# Lecture 11 - Placeholders


```{contents}
:local:
:depth: 1
```
## Learning goals

- 
  [![Colab](https://img.shields.io/badge/Open-Colab-orange)](https://colab.research.google.com/drive/1UYUb5xw7lxDQrZYjJPixfILLrhFCYBlb?usp=sharing)